# [Dijkstra’s Algorithm](https://www.udacity.com/blog/2021/10/implementing-dijkstras-algorithm-in-python.html)

In 1956, Dutch programmer Edsger W. Dijkstra had a practical question. He wanted to figure out the shortest way to travel from Rotterdam to Groningen. But he did not simply consult a map to calculate the distances of the roads he would need to take. Instead, Dijkstra took a computer scientist’s approach: he abstracted from the problem by filtering out the specifics such as traveling from city A to city B. This allowed him to discover the more general problem of graph search. Thus, Dijkstra’s algorithm was born.

Dijkstra’s algorithm is a popular search algorithm used to determine the shortest path between two nodes in a graph. In the original scenario, the graph represented the Netherlands, the graph’s nodes represented different Dutch cities, and the edges represented the roads between the cities. 

You can apply Dijkstra’s algorithm to any problem that can be represented as a graph. Friend suggestions on social media, routing packets over the internet, or finding a way through a maze—the algorithm can do it all. But how does it actually work?

First, we’ll create the Graph class. This class does not cover any of the Dijkstra algorithm’s logic, but it will make the implementation of the algorithm more succinct. 

We’ll implement the graph as a Python dictionary. The dictionary’s keys will correspond to the cities and its values will correspond to dictionaries that record the distances to other cities in the graph. 

In [27]:
import sys


class Graph(object):
    def __init__(self, nodes, init_graph):
        self.nodes = nodes
        self.graph = self.construct_graph(nodes, init_graph)

    def construct_graph(self, nodes, init_graph):
        '''
        This method makes sure that the graph is symmetrical. In other words, if there's a path from node A to B with a value V, there needs to be a path from node B to node A with a value V.
        '''
        graph = {}
        for node in nodes:
            graph[node] = {}

        graph.update(init_graph)

        for node, edges in graph.items():
            for adjacent_node, value in edges.items():
                if graph[adjacent_node].get(node, False) == False:
                    graph[adjacent_node][node] = value

        return graph

    def get_nodes(self):
        "Returns the nodes of the graph."
        return self.nodes

    def get_outgoing_edges(self, node):
        "Returns the neighbors of a node."
        connections = []
        for out_node in self.nodes:
            if self.graph[node].get(out_node, False) != False:
                connections.append(out_node)
        return connections

    def value(self, node1, node2):
        "Returns the value of an edge between two nodes."
        return self.graph[node1][node2]


In [28]:

def dijkstra_algorithm(graph, start_node):
    unvisited_nodes = list(graph.get_nodes())

    # We'll use this dict to save the cost of visiting each node and update it as we move along the graph
    shortest_path = {}

    # We'll use this dict to save the shortest known path to a node found so far
    previous_nodes = {}

    # We'll use max_value to initialize the "infinity" value of the unvisited nodes
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
    # However, we initialize the starting node's value with 0
    shortest_path[start_node] = 0

    # The algorithm executes until we visit all nodes
    while unvisited_nodes:
        # The code block below finds the node with the lowest score
        current_min_node = None
        for node in unvisited_nodes:  # Iterate over the nodes
            if current_min_node == None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node

        # The code block below retrieves the current node's neighbors and updates their distances
        neighbors = graph.get_outgoing_edges(current_min_node)
        for neighbor in neighbors:
            tentative_value = shortest_path[current_min_node] + \
                graph.value(current_min_node, neighbor)
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                # We also update the best path to the current node
                previous_nodes[neighbor] = current_min_node

        # After visiting its neighbors, we mark the node as "visited"
        unvisited_nodes.remove(current_min_node)

    return previous_nodes, shortest_path


Initialize the nodes:

In [29]:
nodes = ["Reykjavik", "Oslo", "Moscow", "London",
         "Rome", "Berlin", "Belgrade", "Athens"]

init_graph = {}
for node in nodes:
    init_graph[node] = {}

init_graph["Reykjavik"]["Oslo"] = 5
init_graph["Reykjavik"]["London"] = 4
init_graph["Oslo"]["Berlin"] = 1
init_graph["Oslo"]["Moscow"] = 3
init_graph["Moscow"]["Belgrade"] = 5
init_graph["Moscow"]["Athens"] = 4
init_graph["Athens"]["Belgrade"] = 1
init_graph["Rome"]["Berlin"] = 2
init_graph["Rome"]["Athens"] = 2


In [35]:
# We’ll use these values to create an object of the Graph class.
graph = Graph(nodes, init_graph)
graph.get_nodes()
graph.get_outgoing_edges("Moscow")

['Oslo', 'Belgrade', 'Athens']

In [31]:
# With our graph fully constructed, we can pass it to the dijkstra_algorithm() function.
previous_nodes, shortest_path = dijkstra_algorithm(
    graph=graph, start_node="Reykjavik")
previous_nodes, shortest_path

({'Oslo': 'Reykjavik',
  'London': 'Reykjavik',
  'Moscow': 'Oslo',
  'Berlin': 'Oslo',
  'Rome': 'Berlin',
  'Belgrade': 'Athens',
  'Athens': 'Rome'},
 {'Reykjavik': 0,
  'Oslo': 5,
  'Moscow': 8,
  'London': 4,
  'Rome': 8,
  'Berlin': 6,
  'Belgrade': 11,
  'Athens': 10})

In [32]:
def print_result(previous_nodes, shortest_path, start_node, target_node):
    path = []
    node = target_node

    while node != start_node:
        path.append(node)
        node = previous_nodes[node]

    # Add the start node manually
    path.append(start_node)

    print("We found the following best path with a value of {}.".format(
        shortest_path[target_node]))
    print(" -> ".join(reversed(path)))


In [33]:

print_result(previous_nodes, shortest_path,
             start_node="Reykjavik", target_node="Belgrade")


We found the following best path with a value of 11.
Reykjavik -> Oslo -> Berlin -> Rome -> Athens -> Belgrade
